In [1]:
import json
import os
import cv2
import numpy as np
from copy import deepcopy
import statistics

In [6]:
f = open(os.path.join('data-apollocar3d', 'annotations', '24kptreg_train.json'))
data = json.load(f)
list(data.keys())

['info', 'categories', 'images', 'annotations']

In [32]:
internals = {'front_lplate': [58, 59, 60, 61], 'rear_lplate': [62,63,64,65], 'right_rear_car_light': [31,32,34,35], 'left_rear_car_light': [22,23,25,26], 'left_front_car_light': [0,1,2,3] , 'right_front_car_light': [54,55,56,57]}
list(zip(data['categories'][0]['keypoints'], list(range(len(data['categories'][0]['keypoints']))))), len(data['images']), len(data['annotations']), len(data['categories'][0]['keypoints'])

([('top_left_c_left_front_car_light', 0),
  ('bottom_left_c_left_front_car_light', 1),
  ('top_right_c_left_front_car_light', 2),
  ('bottom_right_c_left_front_car_light', 3),
  ('top_right_c_left_front_fog_light', 4),
  ('bottom_right_c_left_front_fog_light', 5),
  ('front_section_left_front_wheel', 6),
  ('center_left_front_wheel', 7),
  ('top_right_c_front_glass', 8),
  ('top_left_c_left_front_door', 9),
  ('bottom_left_c_left_front_door', 10),
  ('top_right_c_left_front_door', 11),
  ('middle_c_left_front_door', 12),
  ('front_c_car_handle_left_front_door', 13),
  ('rear_c_car_handle_left_front_door', 14),
  ('bottom_right_c_left_front_door', 15),
  ('top_right_c_left_rear_door', 16),
  ('front_c_car_handle_left_rear_door', 17),
  ('rear_c_car_handle_left_rear_door', 18),
  ('bottom_right_c_left_rear_door', 19),
  ('center_left_rear_wheel', 20),
  ('rear_section_left_rear_wheel', 21),
  ('top_left_c_left_rear_car_light', 22),
  ('bottom_left_c_left_rear_car_light', 23),
  ('top_lef

In [34]:

for annotation in data['annotations']:
    print(len([keypoint for keypoint in annotation['keypoints'] if keypoint != 0]), annotation['num_keypoints'])

6 5
36 30
45 18
6 6
45 18
45 32
45 17
45 0
45 0
45 24
45 18
6 6
36 27
45 32
33 13
30 11
30 12
21 14
30 12
30 15
30 14
30 0
30 0
15 9
45 33
12 10
42 20
45 28
45 18
30 14
27 18
30 16
30 12
30 0
30 0
30 0
30 0
30 0
36 27
0 13
45 20
45 20
0 13
45 31
27 11
30 21
30 0
45 32
45 20
36 29
27 19
30 16
45 18
45 30
6 16
45 18
45 0
45 32
6 14
21 23
45 32
21 25
21 0
21 0
21 0
15 8
12 20
15 6
45 33
15 13
21 11
12 10
15 7
36 22
48 30
27 14
36 27
15 8
45 20
45 0
45 0
45 0
36 14
21 23
36 29
45 32
45 0
45 0
45 0
45 0
45 31
42 33
30 12
45 20
15 13
45 30
45 0
45 0
45 0
45 20
45 31
45 32
45 32
30 14
18 21
18 0
18 0
18 0
36 27
12 6
45 32
3 5
45 18
15 15
33 28
33 0
33 0
45 32
15 14
45 19
45 18
45 16
33 23
45 28
45 0
45 0
45 0
45 0
45 0
15 21
45 31
45 32
45 29
12 13
45 19
15 23
30 10
30 15
21 23
21 0
21 0
21 0
36 23
30 12
30 25
45 30
45 19
45 23
45 27
45 32
45 0
45 0
45 0
45 32
45 22
45 18
15 16
45 22
45 32
45 0
42 30
45 19
15 4
45 17
30 22
6 7
36 16
45 18
45 0
36 12
30 11
15 4
30 16
36 16
45 26
15 5
36 30
21 

In [26]:
# bbox: top left, top right, height, width
# keypoints: rearrange them into a N x 3 table (30 x 3)
# first two columns: x,y coordinate
# Each internal can be represented by a 5x3 table, 6 different internals which gives us 30x3, the fifth row gives us the coordinate for the center
# last column: if it's visible

{'image_id': 34636094, 'category_id': 1, 'iscrowd': 0, 'id': 346360946, 'area': 494, 'bbox': [1921, 1806, 26, 19], 'num_keypoints': 5, 'keypoints': [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1924.99, 1807.82, 2.0, 1923.77, 1813.15, 2.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], 'segmentation': []}
{'image_id': 34636094, 'category_id': 1, 'iscrowd': 0, 'id': 346360940, 'area': 401620, 'bbox': [2207, 1878, 860, 467], 'num_keypoints': 30, 'keypoints': [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 2514.49, 2046.52, 2.0, 2553.8, 2107.18, 2.0, 2604.87, 2060.98, 2.0, 2608.51, 2115.08, 2.0, 2561.5, 2080.8, 2.0, 2980.41, 2128.78, 2.0,

In [35]:
f = open(os.path.join('data-apollocar3d', 'annotations', 'apollo_test_24_modkeypoints.json'))
data = json.load(f)

In [36]:
data['annotations']

[{'image_id': 34630104,
  'category_id': 1,
  'iscrowd': 0,
  'id': 346301044,
  'area': 286482,
  'bbox': [433, 1873, 718, 399],
  'num_keypoints': 28,
  'keypoints': [0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   528.55,
   2015.89,
   2.0,
   493.514,
   2088.78,
   2.0,
   545.464,
   2015.89,
   2.0,
   528.147,
   2096.03,
   2.0,
   847.721,
   2119.46,
   2.0,
   873.481,
   2029.08,
   2.0,
   893.46,
   2113.71,
   2.0,
   901.178,
   2035.86,
   2.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   630.077,
   2058.07,
   2.0,
   750.117,
   2064.45,
   2.0,
   743.158,
   2103.88,
   2.0,
   617.752,
   2100.31,
   2.0],
  'segmentation': []},
 {'image_id': 34630104,
  'category_id': 1,
  'iscrowd': 0,
  'id': 346301047,
  'area': 7310,
  'bbox': [1676, 1752, 86, 85],
  'num_keypoi

In [2]:
# use apollo_train_24_modkeypoint.json, apollo_val_24_modkeypoints.json, apollo_json_test.json as targets
# use images_split folder for loading the images
train_file = open(os.path.join('data-apollocar3d', 'annotations', '24kptreg_train.json'))
data_train = json.load(train_file)
annotations = data_train['annotations']

In [3]:
def find_corresponding_image(image_id, image_dir):

    for img in os.listdir(image_dir):
        if img.find(image_id) != -1:
            return os.path.join(image_dir, img)
    return None

train_dir = os.path.join('data-apollocar3d', 'images_split', 'train')

In [13]:
# bbox: top left, top right, height, width

for annotation in annotations[1000:1025]:

    matching_image = find_corresponding_image(str(annotation['image_id']), train_dir)
    matching_image = cv2.imread(matching_image)

    # bounding box for the vehicle
    bbox = annotation['bbox']

    # draw the box around the vehicle
    cropped_image = deepcopy(matching_image[bbox[1]:bbox[1]+bbox[3], bbox[0]:bbox[0]+bbox[2]])
    matching_image = cv2.rectangle(matching_image, (bbox[0], bbox[1]), (bbox[0] + bbox[2], bbox[1] + bbox[3]), color=(0,0,255), thickness=3)

    if annotation['num_keypoints'] == 0:
        continue

    # Keypoints are in the order left_front_car_light, left_rear_car_light, right_rear_car_light, right_front_car_light, front_lplate, rear_lplate
    keypoints = np.reshape(np.array(annotation['keypoints']), (6,5,3))

    for keypoint_matrix in keypoints:

        for keypoint_loc in keypoint_matrix:

            # If the keypoint isn't visible then
            if keypoint_loc[2] != 0:

                # Draw the keypoint on the image
                # cropped_image= cv2.circle(cropped_image, (int(keypoint_loc[0])-bbox[0], int(keypoint_loc[1])-bbox[1]), radius=5, color=(255,0,0), thickness=-1)

                if int(keypoint_loc[0])-bbox[0] > cropped_image.shape[1]:
                    print('x coordinate of keypoint out of range')
                    print(bbox, keypoint_loc, keypoint_loc[0]-bbox[0], cropped_image.shape)
                    matching_image= cv2.circle(matching_image, (int(keypoint_loc[0]), int(keypoint_loc[1])), radius=10, color=(0,255,0), thickness=-1)
                elif int(keypoint_loc[1])-bbox[1] > cropped_image.shape[0]:
                    print('y coordinate of keypoint out of range')
                    print(bbox, keypoint_loc, keypoint_loc[1]-bbox[1], cropped_image.shape)
                    matching_image= cv2.circle(matching_image, (int(keypoint_loc[0]), int(keypoint_loc[1])), radius=10, color=(0,255,0), thickness=-1)
                else:
                    matching_image= cv2.circle(matching_image, (int(keypoint_loc[0]), int(keypoint_loc[1])), radius=5, color=(255,0,0), thickness=-1)

    print(cropped_image.shape)
    resize_matching_image = cv2.resize(matching_image, (1024,1024))
    cv2.imshow('image', resize_matching_image)
    cv2.imshow('cropped vehicle', cropped_image)
    cv2.waitKey(0)
    cv2.destroyAllWindows()



(56, 169, 3)
(458, 942, 3)
(123, 184, 3)
(50, 46, 3)
(86, 168, 3)
(40, 38, 3)
(167, 310, 3)
(37, 7, 3)
(54, 136, 3)
(55, 96, 3)
(84, 102, 3)
(96, 69, 3)
(180, 276, 3)
(57, 44, 3)
(70, 99, 3)
(112, 150, 3)
(443, 1003, 3)
(58, 354, 3)
(128, 288, 3)
(47, 43, 3)
(61, 77, 3)
(115, 199, 3)
(64, 88, 3)
(78, 51, 3)


In [42]:
avg_vehicle_size = statistics.mean([annotation['area'] for annotation in annotations])
avg_vehicle_size

31999.015398348583

In [38]:
for annotation in annotations[1000:1025]:

    matching_image = find_corresponding_image(str(annotation['image_id']), train_dir)
    matching_image = cv2.imread(matching_image)
    # bounding box for the vehicle
    bbox = annotation['bbox']

    # Keypoints are in the order left_front_car_light, left_rear_car_light, right_rear_car_light, right_front_car_light, front_lplate, rear_lplate
    keypoints = np.reshape(np.array(annotation['keypoints']), (6,5,3))

    for keypoint_matrix in keypoints:

        center_keypoint = keypoint_matrix[-1]
        cropped_image = deepcopy(matching_image[bbox[1]:bbox[1]+bbox[3], bbox[0]:bbox[0]+bbox[2]])

        # If the center of the internal is visible then show an image that centers the internal
        if center_keypoint[2] != 0:


            keypoint_loc = keypoint_matrix[4]

            # Draw the keypoint on the image
            annotated_image = deepcopy(cv2.circle(cropped_image, (int(keypoint_loc[0])-bbox[0], int(keypoint_loc[1])-bbox[1]), radius=5, color=(255,0,0), thickness=-1))


            # The translated coordinates of the center internal in the cropped vehicle image
            translated_keypoint_x, translated_keypoint_y = int(keypoint_loc[0])-bbox[0], int(keypoint_loc[1]) - bbox[1]

            # Crop the image around the internal
            top_left_x = max(translated_keypoint_x-50, 0)
            padding_left = max(50-translated_keypoint_x, 0)
            top_left_y = max(translated_keypoint_y-50, 0)
            padding_top = max(50-translated_keypoint_y, 0)
            bot_right_x = min(translated_keypoint_x+50, cropped_image.shape[1])
            padding_right = max(translated_keypoint_x + 50 - cropped_image.shape[1], 0)
            bot_right_y = min(translated_keypoint_y + 50, cropped_image.shape[0])
            padding_bottom = max(translated_keypoint_y + 50 - cropped_image.shape[0], 0)
            cropped_internal = deepcopy(annotated_image[top_left_y:bot_right_y, top_left_x: bot_right_x])

            if cropped_internal.shape[1] != 0 or cropped_internal.shape[0] != 0:
                padded_image = cv2.copyMakeBorder(cropped_internal, padding_top, padding_bottom, padding_left, padding_right, cv2.BORDER_CONSTANT, None, [0,0,0])
                print(padded_image.shape)
                cv2.imshow('cropped internal', padded_image)
                cv2.waitKey(0)
                cv2.destroyAllWindows()




(100, 100, 3)
(100, 100, 3)
(100, 100, 3)
(100, 100, 3)
(100, 100, 3)
(100, 100, 3)
(100, 100, 3)
(100, 100, 3)
(100, 100, 3)
(100, 100, 3)
(100, 100, 3)
(100, 100, 3)
(100, 100, 3)
(100, 100, 3)
(100, 100, 3)
(100, 100, 3)
(100, 100, 3)
(100, 100, 3)
(100, 100, 3)
(100, 100, 3)
(100, 100, 3)
(100, 100, 3)
(100, 100, 3)
(100, 100, 3)
(100, 100, 3)
(100, 100, 3)
(100, 100, 3)
(100, 100, 3)
(100, 100, 3)
(100, 100, 3)
(100, 100, 3)
(100, 100, 3)
(100, 100, 3)
